In [1]:
!pip install implicit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from implicit.als import AlternatingLeastSquares
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import kagglehub
import os

# Download dataset
path = kagglehub.dataset_download("undefinenull/million-song-dataset-spotify-lastfm")
print("Path to dataset files:", path)

# List available files
print("\nAvailable files:")
for f in os.listdir(path):
    print(f"  • {f}")

# Load FULL user listening history (user_id, track_id, playcount)
print("\nLoading user listening history...")
listening_history = pd.read_csv(
    '/kaggle/input/million-song-dataset-spotify-lastfm/User Listening History.csv'
)

# Keep only: user_id, track_id, playcount
listening_history = listening_history[['user_id', 'track_id', 'playcount']]

print(f"Listening history loaded: {len(listening_history):,} rows")
print(f"Unique users: {listening_history['user_id'].nunique():,}")
print(f"Unique tracks: {listening_history['track_id'].nunique():,}")

# Load track metadata
print("\nLoading track metadata...")
track_metadata = pd.read_csv('/kaggle/input/million-song-dataset-spotify-lastfm/Music Info.csv')

# Keep only: track_id, name, artist, tags
track_metadata = track_metadata[['track_id', 'name', 'artist', 'tags']]

print(f"Track metadata loaded: {len(track_metadata):,} tracks")
print(f"Columns: {list(track_metadata.columns)}")

print(f"\n✓ Both datasets loaded!")
print(f"\nListening history:")
print(listening_history.head())
print(f"\nTrack metadata:")
print(track_metadata.head())

# Prepare for training
music_df = listening_history.rename(columns={'track_id': 'artist_id', 'playcount': 'play_count'})

unique_users = music_df['user_id'].unique()
user_id_map = {user: idx for idx, user in enumerate(unique_users)}
music_df['user_id'] = music_df['user_id'].map(user_id_map)

print(f"\n✓ Last.fm preprocessed!")
print(f"  Users: {music_df['user_id'].nunique():,}")
print(f"  Tracks: {music_df['artist_id'].nunique():,}")
print(f"  Interactions: {len(music_df):,}")


Using Colab cache for faster access to the 'million-song-dataset-spotify-lastfm' dataset.
Path to dataset files: /kaggle/input/million-song-dataset-spotify-lastfm

Available files:
  • MP3-Example
  • Music Info.csv
  • User Listening History.csv

Loading user listening history...
Listening history loaded: 9,711,301 rows
Unique users: 962,037
Unique tracks: 30,459

Loading track metadata...
Track metadata loaded: 50,683 tracks
Columns: ['track_id', 'name', 'artist', 'tags']

✓ Both datasets loaded!

Listening history:
                                    user_id            track_id  playcount
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  TRIRLYL128F42539D1          1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  TRFUPBA128F934F7E1          1
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  TRLQPQJ128F42AA94F          1
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  TRTUCUY128F92E1D24          1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  TRHDDQG12903CB53EE          1

Track metadata:
         

In [5]:
# Set the path to the file you'd like to load
file_path = "rating.csv"

# Load the latest version
ratings_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "grouplens/movielens-20m-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", ratings_df.head())

/tmp/ipython-input-836257605.py:5: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  ratings_df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'movielens-20m-dataset' dataset.
First 5 records:    userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


In [6]:
# Set the path to the file you'd like to load
file_path = "movie.csv"

# Load the latest version
movies_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "grouplens/movielens-20m-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", movies_df.head())

/tmp/ipython-input-1023958349.py:5: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  movies_df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'movielens-20m-dataset' dataset.
First 5 records:    movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [7]:
# Cell 6: Create Sparse User-Item Matrices

from scipy.sparse import csr_matrix

# MovieLens sparse matrix
movie_user_ids = ratings_df['userId'].astype('category')
movie_item_ids = ratings_df['movieId'].astype('category')

movie_sparse = csr_matrix(
    (ratings_df['rating'], (movie_user_ids.cat.codes, movie_item_ids.cat.codes))
)

print(f"MovieLens sparse matrix shape: {movie_sparse.shape}")
print(f"Sparsity: {100 * (1 - movie_sparse.nnz / (movie_sparse.shape[0] * movie_sparse.shape[1])):.2f}%")

# Music sparse matrix (use playcount as implicit feedback)
music_user_ids = listening_history['user_id'].astype('category')
music_track_ids = listening_history['track_id'].astype('category')

music_sparse = csr_matrix(
    (listening_history['playcount'], (music_user_ids.cat.codes, music_track_ids.cat.codes))
)

print(f"\nMusic sparse matrix shape: {music_sparse.shape}")
print(f"Sparsity: {100 * (1 - music_sparse.nnz / (music_sparse.shape[0] * music_sparse.shape[1])):.2f}%")

# Save the mappings for later use
movie_user_map = dict(enumerate(movie_user_ids.cat.categories))
movie_item_map = dict(enumerate(movie_item_ids.cat.categories))
music_user_map = dict(enumerate(music_user_ids.cat.categories))
music_track_map = dict(enumerate(music_track_ids.cat.categories))


MovieLens sparse matrix shape: (138493, 26744)
Sparsity: 99.46%

Music sparse matrix shape: (962037, 30459)
Sparsity: 99.97%


In [8]:
# Cell 7: Train ALS Models on Both Domains

from implicit.als import AlternatingLeastSquares
import numpy as np

# Initialize ALS models
movie_model = AlternatingLeastSquares(
    factors=50,           # Embedding dimension
    regularization=0.01,  # L2 regularization
    iterations=10,        # Training iterations
    random_state=42
)

music_model = AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=10,
    random_state=42
)

print("Training MovieLens ALS model...")
movie_model.fit(movie_sparse.T)  # Transpose: ALS expects items x users
print("✓ MovieLens model trained")

print("\nTraining Music ALS model...")
music_model.fit(music_sparse.T)  # Transpose: ALS expects items x users
print("✓ Music model trained")

print(f"\nMovie user embeddings shape: {movie_model.user_factors.shape}")
print(f"Movie item embeddings shape: {movie_model.item_factors.shape}")
print(f"Music user embeddings shape: {music_model.user_factors.shape}")
print(f"Music track embeddings shape: {music_model.item_factors.shape}")


/usr/local/lib/python3.12/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Training MovieLens ALS model...


/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.911750078201294 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

✓ MovieLens model trained

Training Music ALS model...


/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.32040953636169434 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

✓ Music model trained

Movie user embeddings shape: (26744, 50)
Movie item embeddings shape: (138493, 50)
Music user embeddings shape: (30459, 50)
Music track embeddings shape: (962037, 50)


In [27]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 18.2 MB/s eta 0:00:00


In [12]:
# Cell 8 (REVISED): Automatic Semantic Mapping using GloVe Embeddings

import gensim.downloader as api
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# First, create the cleaned columns
print("Preprocessing movie genres and music tags...")
movies_df['genres_clean'] = movies_df['genres'].fillna('unknown').str.replace('|', ' ').str.lower()
track_metadata['tags_clean'] = track_metadata['tags'].fillna('unknown').str.replace(',', ' ').str.lower()
print("✓ Preprocessing complete\n")

# Load pre-trained GloVe embeddings (downloads ~100MB first time only)
print("Loading GloVe embeddings...")
glove_model = api.load("glove-wiki-gigaword-100")  # 100-dimensional vectors
print("✓ GloVe loaded\n")

# Extract unique words from both domains
movie_genre_words = set()
for genres in movies_df['genres_clean']:
    movie_genre_words.update(genres.split())

music_tag_words = set()
for tags in track_metadata['tags_clean']:
    music_tag_words.update(tags.split())

print(f"Unique movie genre words: {len(movie_genre_words)}")
print(f"Unique music tag words: {len(music_tag_words)}")

# Helper function to get word embedding
def get_word_embedding(word):
    try:
        return glove_model[word]
    except KeyError:
        return None

# Expand genres with semantically similar music tags
def expand_genre_with_embeddings(genres_str, top_k=10, similarity_threshold=0.5):
    """Find semantically similar music tags for movie genres using GloVe"""
    genre_words = genres_str.split()
    expanded = list(genre_words)  # Keep original words

    for genre_word in genre_words:
        genre_emb = get_word_embedding(genre_word)
        if genre_emb is None:
            continue

        # Compute similarity to all music tags
        similarities = []
        for music_word in music_tag_words:
            music_emb = get_word_embedding(music_word)
            if music_emb is not None:
                sim = cosine_similarity([genre_emb], [music_emb])[0][0]
                if sim > similarity_threshold:
                    similarities.append((music_word, sim))

        # Add top-k most similar music tags
        similarities.sort(key=lambda x: x[1], reverse=True)
        expanded.extend([word for word, sim in similarities[:top_k]])

    return ' '.join(expanded)


# PRE-COMPUTE genre-to-music mapping (do this ONCE)
print("\nPre-computing semantic mappings between genres and music tags...")
genre_to_music_map = {}

for genre_word in movie_genre_words:
    genre_emb = get_word_embedding(genre_word)
    if genre_emb is None:
        genre_to_music_map[genre_word] = []
        continue

    # Compute similarity to all music tags ONCE per genre word
    similarities = []
    for music_word in music_tag_words:
        music_emb = get_word_embedding(music_word)
        if music_emb is not None:
            sim = cosine_similarity([genre_emb], [music_emb])[0][0]
            if sim > 0.5:  # Threshold
                similarities.append((music_word, sim))

    # Store top 10 matches
    similarities.sort(key=lambda x: x[1], reverse=True)
    genre_to_music_map[genre_word] = [word for word, sim in similarities[:10]]

    print(f"  {genre_word} → {genre_to_music_map[genre_word][:5]}...")  # Show progress

print(f"\n✓ Mapping complete for {len(genre_to_music_map)} genre words\n")

# Now use the pre-computed mapping (FAST)
def expand_genre_fast(genres_str):
    """Expand using pre-computed mapping"""
    genre_words = genres_str.split()
    expanded = list(genre_words)

    for genre_word in genre_words:
        if genre_word in genre_to_music_map:
            expanded.extend(genre_to_music_map[genre_word])

    return ' '.join(expanded)

# Apply expansion (now very fast!)
print("Expanding movie genres with pre-computed mappings...")
movies_df['genres_expanded'] = movies_df['genres_clean'].apply(expand_genre_fast)
print("✓ Expansion complete\n")



print("\n" + "="*80)
print("Example semantic expansions:")
print("="*80)
for i in range(5):
    print(f"\n{movies_df.iloc[i]['title']}")
    print(f"Original: {movies_df.iloc[i]['genres_clean']}")
    print(f"Expanded: {movies_df.iloc[i]['genres_expanded']}")

# Create TF-IDF vectors with expanded genres
print("\n" + "="*80)
all_movie_text = movies_df['genres_expanded'].tolist()
all_music_text = track_metadata['tags_clean'].tolist()

vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
vectorizer.fit(all_movie_text + all_music_text)

movie_genre_vectors = vectorizer.transform(all_movie_text)
music_tag_vectors = vectorizer.transform(all_music_text)

print(f"Movie genre vectors shape: {movie_genre_vectors.shape}")
print(f"Music tag vectors shape: {music_tag_vectors.shape}")
print(f"Shared vocabulary size: {len(vectorizer.vocabulary_)}")

Preprocessing movie genres and music tags...
✓ Preprocessing complete

Loading GloVe embeddings...
✓ GloVe loaded

Unique movie genre words: 22
Unique music tag words: 101

Pre-computing semantic mappings between genres and music tags...
  children → ['love', 'country', 'american', 'cover']...
  animation → ['soundtrack']...
  comedy → ['soundtrack', 'love', 'dance', 'indie', 'pop']...
  drama → ['love', 'soundtrack', 'dance']...
  war → ['country', 'american', 'british', 'german', 'russian']...
  crime → ['country']...
  documentary → ['soundtrack']...
  western → ['country', 'american', 'british', 'russian', 'french']...
  mystery → ['unknown', 'love']...
  horror → ['soundtrack', 'love', 'indie']...
  genres → ['folk', 'pop', 'classical', 'punk', 'rap']...
  fantasy → ['soundtrack', 'love', 'soul']...
  imax → []...
  adventure → ['love']...
  action → ['american', 'country', 'british']...
  romance → ['love']...
  thriller → ['soundtrack']...
  musical → ['dance', 'folk', 'soundtra

In [13]:
# Cell 9: On-Demand Music Recommendation Function

def recommend_music_for_movie(movie_id, n_recommendations=10):
    """
    Recommend music using on-demand similarity computation
    """
    # Find movie
    movie_idx = movies_df[movies_df['movieId'] == movie_id].index[0]
    movie_title = movies_df.iloc[movie_idx]['title']
    movie_genres = movies_df.iloc[movie_idx]['genres']

    print(f"Movie: {movie_title}")
    print(f"Genres: {movie_genres}\n")

    # Compute similarity ONLY for this one movie (fast!)
    movie_vector = movie_genre_vectors[movie_idx:movie_idx+1]  # Shape: (1, vocab_size)
    similarity_scores = cosine_similarity(movie_vector, music_tag_vectors)[0]  # Shape: (50683,)

    # Get top N
    top_indices = similarity_scores.argsort()[-n_recommendations:][::-1]

    # Create results
    results = []
    for idx in top_indices:
        track_info = track_metadata.iloc[idx]
        results.append({
            'track': track_info['name'],
            'artist': track_info['artist'],
            'tags': track_info['tags'],
            'similarity': similarity_scores[idx]
        })

    return pd.DataFrame(results)


# Test the recommendation system
print("="*80)
print("TEST 1: Toy Story (Animation/Children/Comedy)")
print("="*80)
recs1 = recommend_music_for_movie(movie_id=1, n_recommendations=10)
print(recs1.to_string(index=False))

print("\n" + "="*80)
print("TEST 2: The Dark Knight (Action/Crime/Thriller)")
print("="*80)
# Find The Dark Knight movie ID
dark_knight = movies_df[movies_df['title'].str.contains('Dark Knight', case=False, na=False)]
if len(dark_knight) > 0:
    recs2 = recommend_music_for_movie(movie_id=dark_knight.iloc[0]['movieId'], n_recommendations=10)
    print(recs2.to_string(index=False))
else:
    print("Dark Knight not found, trying another action movie...")
    action_movies = movies_df[movies_df['genres'].str.contains('Action', na=False)]
    recs2 = recommend_music_for_movie(movie_id=action_movies.iloc[0]['movieId'], n_recommendations=10)
    print(recs2.to_string(index=False))


TEST 1: Toy Story (Animation/Children/Comedy)
Movie: Toy Story (1995)
Genres: Adventure|Animation|Children|Comedy|Fantasy

               track               artist                        tags  similarity
         All Is Love Karen O and the Kids            soundtrack, love    0.589862
      From the Heart        Another Level            soundtrack, love    0.589862
     Defying Gravity            Glee Cast     soundtrack, love, cover    0.538139
           If I Fell     Evan Rachel Wood     soundtrack, love, cover    0.538139
    It Won't Be Long     Evan Rachel Wood     soundtrack, love, cover    0.538139
   Look What I Found            Lady Gaga  soundtrack, american, love    0.532859
My One and Only Love        Frank Sinatra                        love    0.482635
     Save Your Heart        Mayday Parade                        love    0.482635
               Amado      Vanessa da Mata                        love    0.482635
       St. Apollonia               Beirut indie, folk, am

In [14]:
# Cell 10: Evaluation Metrics

import numpy as np
from collections import Counter

def evaluate_recommendations(n_test_movies=100, n_recommendations=10):
    """
    Evaluate recommendation quality using:
    - Diversity: How varied are the recommended tracks?
    - Coverage: What % of music catalog is recommended?
    - Genre alignment: Do music tags match movie genres?
    """

    print("="*80)
    print(f"EVALUATING RECOMMENDATIONS FOR {n_test_movies} RANDOM MOVIES")
    print("="*80)

    # Sample random movies
    test_movie_ids = movies_df.sample(n_test_movies, random_state=42)['movieId'].tolist()

    all_recommended_tracks = []
    all_similarity_scores = []
    genre_tag_overlaps = []

    for movie_id in test_movie_ids:
        try:
            recs = recommend_music_for_movie(movie_id, n_recommendations=n_recommendations)
            all_recommended_tracks.extend(recs['track'].tolist())
            all_similarity_scores.extend(recs['similarity'].tolist())

            # Check genre-tag overlap
            movie_idx = movies_df[movies_df['movieId'] == movie_id].index[0]
            movie_genres = set(movies_df.iloc[movie_idx]['genres_clean'].split())

            for _, row in recs.iterrows():
                music_tags = set(row['tags'].lower().split(','))
                overlap = len(movie_genres.intersection(music_tags))
                genre_tag_overlaps.append(overlap)

        except Exception as e:
            continue

    # Compute metrics
    unique_tracks = len(set(all_recommended_tracks))
    total_tracks = len(all_recommended_tracks)
    catalog_size = len(track_metadata)

    print(f"\n📊 DIVERSITY METRICS:")
    print(f"  Unique tracks recommended: {unique_tracks:,}")
    print(f"  Total recommendations: {total_tracks:,}")
    print(f"  Diversity ratio: {unique_tracks/total_tracks:.2%}")

    print(f"\n📊 COVERAGE METRICS:")
    print(f"  Music catalog size: {catalog_size:,}")
    print(f"  Tracks covered: {unique_tracks:,}")
    print(f"  Catalog coverage: {unique_tracks/catalog_size:.2%}")

    print(f"\n📊 SIMILARITY METRICS:")
    print(f"  Mean similarity score: {np.mean(all_similarity_scores):.4f}")
    print(f"  Median similarity score: {np.median(all_similarity_scores):.4f}")
    print(f"  Min similarity score: {np.min(all_similarity_scores):.4f}")
    print(f"  Max similarity score: {np.max(all_similarity_scores):.4f}")

    print(f"\n📊 GENRE-TAG ALIGNMENT:")
    print(f"  Mean genre-tag overlaps: {np.mean(genre_tag_overlaps):.2f}")
    print(f"  Recommendations with overlap: {sum(1 for x in genre_tag_overlaps if x > 0)/len(genre_tag_overlaps):.2%}")

    # Top recommended tracks
    track_counts = Counter(all_recommended_tracks)
    print(f"\n📊 TOP 10 MOST RECOMMENDED TRACKS:")
    for track, count in track_counts.most_common(10):
        print(f"  {track}: {count} times")

    return {
        'diversity': unique_tracks/total_tracks,
        'coverage': unique_tracks/catalog_size,
        'mean_similarity': np.mean(all_similarity_scores),
        'mean_genre_overlap': np.mean(genre_tag_overlaps)
    }

# Run evaluation
eval_results = evaluate_recommendations(n_test_movies=100, n_recommendations=10)


EVALUATING RECOMMENDATIONS FOR 100 RANDOM MOVIES
Movie: Black Caesar (1973)
Genres: Crime|Drama

Movie: G-Force (2009)
Genres: Action|Adventure|Children|Fantasy

Movie: Raze (2013)
Genres: Action|Horror

Movie: Moonwalker (1988)
Genres: Musical

Movie: Don Q Son of Zorro (1925)
Genres: Adventure|Romance

Movie: Chaos (2005)
Genres: Crime|Drama|Horror

Movie: Devil's Own, The (1997)
Genres: Action|Drama|Thriller

Movie: Other Man, The (2008)
Genres: Drama

Movie: Descent (2007)
Genres: Drama|Thriller

Movie: Royal Scandal, The (2001)
Genres: Crime|Mystery

Movie: Jekyll (2007)
Genres: Crime|Horror|Thriller

Movie: Sexual Life (2005)
Genres: Romance

Movie: Darby O'Gill and the Little People (1959)
Genres: Adventure|Children|Fantasy

Movie: Arnold (1973)
Genres: Comedy|Horror

Movie: Songs From the Second Floor (Sånger från andra våningen) (2000)
Genres: Drama

Movie: Dirty Work (1998)
Genres: Comedy

Movie: Amer (2009)
Genres: Drama|Horror

Movie: Four Christmases (2008)
Genres: Comedy


In [15]:
# Cell 11: Save Models and Results

import pickle
import json

print("="*80)
print("SAVING MODELS AND RESULTS")
print("="*80)

# 1. Save the vectorizer and similarity components
print("\n1. Saving TF-IDF vectorizer...")
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
print("  ✓ Saved: tfidf_vectorizer.pkl")

# 2. Save genre-to-music mapping
print("\n2. Saving genre-to-music mapping...")
with open('genre_to_music_map.json', 'w') as f:
    json.dump(genre_to_music_map, f, indent=2)
print("  ✓ Saved: genre_to_music_map.json")

# 3. Save processed dataframes
print("\n3. Saving processed dataframes...")
movies_df.to_csv('movies_processed.csv', index=False)
track_metadata.to_csv('tracks_processed.csv', index=False)
print("  ✓ Saved: movies_processed.csv")
print("  ✓ Saved: tracks_processed.csv")

# 4. Save evaluation results
print("\n4. Saving evaluation results...")
with open('evaluation_results.json', 'w') as f:
    json.dump(eval_results, f, indent=2)
print("  ✓ Saved: evaluation_results.json")

# 5. Generate sample recommendations for report
print("\n5. Generating sample recommendations for report...")
sample_movies = [
    ('Toy Story (1995)', 1),
    ('The Matrix (1999)', movies_df[movies_df['title'].str.contains('Matrix', na=False)].iloc[0]['movieId'] if len(movies_df[movies_df['title'].str.contains('Matrix', na=False)]) > 0 else 1),
    ('Titanic (1997)', movies_df[movies_df['title'].str.contains('Titanic', na=False)].iloc[0]['movieId'] if len(movies_df[movies_df['title'].str.contains('Titanic', na=False)]) > 0 else 1),
]

all_sample_recs = []
for title, movie_id in sample_movies:
    try:
        recs = recommend_music_for_movie(movie_id, n_recommendations=5)
        recs['movie'] = title
        all_sample_recs.append(recs)
    except:
        continue

if all_sample_recs:
    sample_recs_df = pd.concat(all_sample_recs, ignore_index=True)
    sample_recs_df.to_csv('sample_recommendations.csv', index=False)
    print("  ✓ Saved: sample_recommendations.csv")

# 6. Save the ALS models (for future hybrid use)
print("\n6. Saving ALS models...")
with open('movie_als_model.pkl', 'wb') as f:
    pickle.dump(movie_model, f)
with open('music_als_model.pkl', 'wb') as f:
    pickle.dump(music_model, f)
print("  ✓ Saved: movie_als_model.pkl")
print("  ✓ Saved: music_als_model.pkl")

print("\n" + "="*80)
print("✅ ALL MODELS AND RESULTS SAVED SUCCESSFULLY!")
print("="*80)
print("\nSaved files:")
print("  - tfidf_vectorizer.pkl")
print("  - genre_to_music_map.json")
print("  - movies_processed.csv")
print("  - tracks_processed.csv")
print("  - evaluation_results.json")
print("  - sample_recommendations.csv")
print("  - movie_als_model.pkl")
print("  - music_als_model.pkl")


SAVING MODELS AND RESULTS

1. Saving TF-IDF vectorizer...
  ✓ Saved: tfidf_vectorizer.pkl

2. Saving genre-to-music mapping...
  ✓ Saved: genre_to_music_map.json

3. Saving processed dataframes...
  ✓ Saved: movies_processed.csv
  ✓ Saved: tracks_processed.csv

4. Saving evaluation results...
  ✓ Saved: evaluation_results.json

5. Generating sample recommendations for report...
Movie: Toy Story (1995)
Genres: Adventure|Animation|Children|Comedy|Fantasy

Movie: Matrix, The (1999)
Genres: Action|Sci-Fi|Thriller

Movie: Titanic (1997)
Genres: Drama|Romance

  ✓ Saved: sample_recommendations.csv

6. Saving ALS models...
  ✓ Saved: movie_als_model.pkl
  ✓ Saved: music_als_model.pkl

✅ ALL MODELS AND RESULTS SAVED SUCCESSFULLY!

Saved files:
  - tfidf_vectorizer.pkl
  - genre_to_music_map.json
  - movies_processed.csv
  - tracks_processed.csv
  - evaluation_results.json
  - sample_recommendations.csv
  - movie_als_model.pkl
  - music_als_model.pkl
